# SSNE Miniproject 5
### 318703 Tomasz Owienko
### 318718 Anna Schäfer
### Grupa piątek

In [123]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import pickle
from torch.utils.data import DataLoader, Dataset, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [124]:
RANDOM_SEED = 123
pl.seed_everything(RANDOM_SEED)

Seed set to 123


123

In [125]:
device = torch.device("cuda")

In [126]:
VALIDATION_PERCENTAGE = 0.10
batch_size = 8
TRAIN_PATH = "data/train.pkl"
TEST_PATH = "data/test_no_target.pkl"

In [127]:
class VariableLenDataset(Dataset):
    def __init__(self, in_data, target):
        self.data = [(x, y) for x, y in zip(in_data, target)]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        in_data, target = self.data[idx]
        return in_data, target

In [128]:
def pad_collate(batch):
    (xx, yy) = zip(*batch)
    xx_pad = nn.utils.rnn.pad_sequence(xx, batch_first=True, padding_value=0)
    xx_pad = xx_pad.unsqueeze(-1)
    return xx_pad, torch.tensor(yy, dtype=torch.long)

In [129]:
with open(TRAIN_PATH, 'rb') as f:
    train = pickle.load(f)

train_data = [[torch.from_numpy(t[0].astype(int)).float(), int(t[1])] for t in train]

In [130]:
dataset_length = len(train_data)
val_size = int(dataset_length * VALIDATION_PERCENTAGE)
train_size = dataset_length - val_size
train_subset, val_subset = random_split(train_data, [train_size, val_size])

train_dataset = VariableLenDataset([x[0] for x in train_subset], [x[1] for x in train_subset])
val_dataset = VariableLenDataset([x[0] for x in val_subset], [x[1] for x in val_subset])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=pad_collate)

In [131]:
class LSTMClassifier(pl.LightningModule):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        # Initialize hidden state with correct dimensions
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)
        loss = self.criterion(outputs, y)
        acc = (outputs.argmax(dim=1) == y).float().mean()
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)
        loss = self.criterion(outputs, y)
        acc = (outputs.argmax(dim=1) == y).float().mean()
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [132]:
input_size = train_data[0][0].shape[1] if len(train_data[0][0].shape) > 1 else 1
hidden_size = 128
output_size = len(set(x[1] for x in train_data))

model = LSTMClassifier(input_size=input_size, hidden_size=hidden_size, output_size=output_size)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='model_checkpoints',
    filename='best_model',
    save_top_k=1,
    mode='min'
)

trainer = pl.Trainer(max_epochs=20, callbacks=[checkpoint_callback])
trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\annak\PycharmProjects\24l-ssne\.venv\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:653: Checkpoint directory C:\Users\annak\PycharmProjects\24l-ssne\proj5\model_checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | lstm      | LSTM             | 67.1 K
1 | fc        | Linear           | 645   
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
67.7 K    Trainable params
0         Non-trainable params
67.7 K    Total params
0.271     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
with open(TEST_PATH, 'rb') as f:
    test_data = pickle.load(f)

test_data = [torch.from_numpy(t.astype(int)).float().unsqueeze(-1) for t in test_data]
test_dataset = VariableLenDataset(test_data, [0 for _ in test_data])
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False, collate_fn=pad_collate)